# Recommend movie lists using user preferences and item content data

Importing all the library

In [56]:
import pandas as pd
import numpy as np
import random

from surprise import SVD, SVDpp
from surprise import Dataset
from surprise import Reader
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNBaseline
from surprise import KNNWithZScore
from surprise import CoClustering
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise import accuracy

from tabulate import tabulate

from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold

Defining the path of the data

In [28]:
path = '../Data/train-PDA2019.csv'
path_2 = '../Data/test-PDA2019.csv'

Explore the data

In [29]:
ratings_df = pd.read_csv(path)
ratings_df.coloums = ['userID', 'itemID','rating','timeStamp']
ratings_df.head()

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


,userID,itemID,rating,timeStamp
0,5,648,5,978297876
1,5,1394,5,978298237
2,5,3534,5,978297149
3,5,104,4,978298558
4,5,2735,5,978297919


In [30]:
ratings_df.describe()

,userID,itemID,rating,timeStamp
count,470711.000000,470711.000000,470711.000000,4.707110e+05
mean,5982.290023,1928.312204,3.638362,9.724536e+08
std,3479.194361,1046.635193,1.100136,1.235020e+07
min,1.000000,89.000000,1.000000,9.567040e+08
25%,2901.000000,1097.000000,3.000000,9.653109e+08
50%,5993.000000,1945.000000,4.000000,9.733477e+08
75%,8913.000000,2797.000000,4.000000,9.753465e+08
max,12079.000000,3952.000000,5.000000,1.046438e+09


In [47]:
test_df = pd.read_csv(path_2)
test_df.coloums = ['userID', 'recommended_itemIDs']

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [48]:
test_df.head()

,userID,recommended_itemIDs
0,1,
1,3,
2,11,
3,29,
4,31,


Let's run a test for finding which of the method saw in class could be used

In [7]:
recommenders = (SVD, KNNBasic, BaselineOnly, KNNWithZScore, CoClustering, NormalPredictor)
titles = ('SVD','KNN-Basic','KNN-Baseline','KNNWithZScore','CoClustering','Baseline','Random')

In [17]:
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(ratings_df[['userID', 'itemID','rating']], reader)

In [9]:
kf = KFold(random_state=0)

table = []
fold_n = 0
for rec in recommenders:
    out = cross_validate(rec(), data, ['rmse','mae','fcp'],kf)
    mean_rmse = '{:.3f}'.format(np.mean(out['test_rmse']))
    mean_mae = '{:.3f}'.format(np.mean(out['test_mae']))
    mean_fcp = '{:.3f}'.format(np.mean(out['test_fcp']))
    
    new_line = [titles[fold_n], mean_rmse, mean_mae, mean_fcp]
    table.append(new_line)
    fold_n += 1 

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


In [10]:
header = ['Recommenders','Pred Accuracy (RMSE)','Pred Accuracy (MAE)', 'Rank quality (FCP)']

print(tabulate(table, header, tablefmt='pipe'))

| Recommenders   |   Pred Accuracy (RMSE) |   Pred Accuracy (MAE) |   Rank quality (FCP) |
|:---------------|-----------------------:|----------------------:|---------------------:|
| SVD            |                  0.886 |                 0.695 |                0.72  |
| KNN-Basic      |                  0.932 |                 0.731 |                0.715 |
| KNN-Baseline   |                  0.903 |                 0.713 |                0.707 |
| KNNWithZScore  |                  0.929 |                 0.734 |                0.704 |
| CoClustering   |                  0.913 |                 0.714 |                0.712 |
| Baseline       |                  1.484 |                 1.187 |                0.495 |


By looking to the test, the best model to use it SVD, let's implement it